<a href="https://colab.research.google.com/github/ksc3045/NLP-Assignments/blob/main/NLP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLP in practice: Spam filtering**

## **Machine learning pipeline**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Academic/2025/January - 25/NLP/Lectures/Spam vs Ham
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Academic/2025/January - 25/NLP/Lectures/Spam vs Ham
Chapter2.ipynb  enron1/  enron1.zip


Read in spam and ham file lists:

In [ ]:
import os   # Using os functionality, list all the files in the specified folder.
import codecs # helps with different text encodings.

def read_in(folder):
    files = os.listdir(folder)
    a_list = []
    for a_file in files:      # Iterate through the files in the folder.
        if not a_file.startswith("."):    # Skip hidden files
            # Read the contents of each file.
            f = codecs.open(folder + a_file, "r", encoding = "ISO-8859-1", errors="ignore")
            # Add the content of each file to the list data structure.
            a_list.append(f.read())
            f.close() # Close the file after you read the contents.
    return a_list   # Return Python list that contains the contents of the files from the specified folder.

Initialise lists and print out length – this should return 1500 for `enron1/spam` and 3672 for `enron1/ham`:

In [ ]:
spam_list = read_in("enron1/spam/")
print(len(spam_list))
# Print out the contents of the first entry (i.e., the first file in each correspondent subfolder).
print(spam_list[0])

ham_list = read_in("enron1/ham/")
print(len(ham_list))
# Print out the contents of the first entry (i.e., the first file in each correspondent subfolder).
print(ham_list[0])

1500
Subject: holiday voucher #608324 tropical travel package
You are receiving this email because you opted in at one of our
Associates. Please do not reply to this email as this address does not
Accept mail. To remove yourself from this list and not receive
Future offersplease click
Here, or send a request (including email address) to:
Voucher fulfillmentl 001 cypress creek road, suite 316 fort
Lauderdale, fl 33309
3672
Subject: sitara position
Please use the position manager home button.
When using the sitara position manager, and you drill down to view details
Or change the dates, the position manager keeps
All your views running in the background. This feature allows you to use the
Back and forward buttons to keep your views running.
On the downside, if you keep these views running but do not really intend to
Go back to look at them, the system and your pc are working harder
To keep all the old views active and updated.
The home button sends you back to your default view - the fir

In [ ]:
import random

all_emails = [(email_content, "spam") for email_content in spam_list]
all_emails += [(email_content, "ham") for email_content in ham_list]
random.seed(42)
random.shuffle(all_emails)
print (f"Dataset size = {str(len(all_emails))} emails")

Dataset size = 5172 emails


### Step 2: Split the text into words

Preprocess the texts by tokenising them and removing the stopwords

In [ ]:
# This code is important otherwise NLTK library was producing errors
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import nltk
from nltk import word_tokenize

def get_features(text):
    features = {}
    word_list = [word for word in word_tokenize(text.lower())]
    for word in word_list:
        features[word] = True
    return features

all_features = [(get_features(email), label) for (email, label) in all_emails]

print(get_features("Participate In Our New Lottery NOW!"))
print(len(all_features))
print(len(all_features[0][0]))
print(len(all_features[99][0]))

### Step 3: Extract and normalize the features



### Step 4: Train a classifier




Apply Naive Bayes classifier:

In [ ]:
from nltk import NaiveBayesClassifier, classify

def train(features, proportion):
    train_size = int(len(features) * proportion)
    # initialise the training and test sets
    train_set, test_set = features[:train_size], features[train_size:]
    print (f"Training set size = {str(len(train_set))} emails")
    print (f"Test set size = {str(len(test_set))} emails")
    # train the classifier
    classifier = NaiveBayesClassifier.train(train_set)
    return train_set, test_set, classifier

train_set, test_set, classifier = train(all_features, 0.8)

### Step 5: Evaluate the classifier

In [ ]:
def evaluate(train_set, test_set, classifier):
    # check how the classifier performs on the training and test sets
    print (f"Accuracy on the training set = {str(classify.accuracy(classifier, train_set))}")
    print (f"Accuracy on the test set = {str(classify.accuracy(classifier, test_set))}")
    # check which words are most informative for the classifier
    classifier.show_most_informative_features(50)

evaluate(train_set, test_set, classifier)

Explore the contexts of use:

In [ ]:
from nltk.text import Text

def concordance(data_list, search_word):
    for email in data_list:
        word_list = [word for word in word_tokenize(email.lower())]
        text_list = Text(word_list)
        if search_word in word_list:
            text_list.concordance(search_word)


print ("STOCKS in HAM:")
concordance(ham_list, "stocks")
print ("\n\nSTOCKS in SPAM:")
concordance(spam_list, "stocks")

Input some of your own messages:

In [ ]:
test_spam_list = ["Participate in our new lottery!", "Try out this new medicine"]
test_ham_list = ["See the minutes from the last meeting attached",
                 "Investors are coming to our office on Monday"]

test_emails = [(email_content, "spam") for email_content in test_spam_list]
test_emails += [(email_content, "ham") for email_content in test_ham_list]

new_test_set = [(get_features(email), label) for (email, label) in test_emails]

evaluate(train_set, new_test_set, classifier)

Accuracy on the training set = 0.9659173313995649
Accuracy on the test set = 1.0
Most Informative Features
                    2004 = True             spam : ham    =    130.9 : 1.0
            prescription = True             spam : ham    =    121.3 : 1.0
                     nom = True              ham : spam   =    121.1 : 1.0
                    pain = True             spam : ham    =     97.2 : 1.0
                  farmer = True              ham : spam   =     86.7 : 1.0
                     sex = True             spam : ham    =     85.9 : 1.0
                    spam = True             spam : ham    =     85.9 : 1.0
                    2001 = True              ham : spam   =     77.9 : 1.0
                     ect = True              ham : spam   =     75.0 : 1.0
                  weight = True             spam : ham    =     74.7 : 1.0
                  differ = True             spam : ham    =     69.9 : 1.0
              nomination = True              ham : spam   =     68.8

See how they get classified:

In [ ]:
for email in test_spam_list:
    print (email)
    print (classifier.classify(get_features(email)))
for email in test_ham_list:
    print (email)
    print (classifier.classify(get_features(email)))

Participate in our new lottery!
spam
Try out this new medicine
spam
See the minutes from the last meeting attached
ham
Investors are coming to our office on Monday
ham


Run in an interactive manner:

In [ ]:
while True:
    email = input("Type in your email here (or press 'Enter'): ")
    if len(email)==0:
        break
    else:
        prediction = classifier.classify(get_features(email))
        print (f"This email is likely {prediction}\n")

Type in your email here (or press 'Enter'): 


Run on a different dataset:

# Assignment:

Apply the classifier to a different test set, e.g. the emails from `enron2/`. As before, you need to read in the data, extract textual content, extract the features and evaluate the classifier.

One of such publicly available collections is Enron email dataset. (You can read more about the dataset at www.cs.cmu.edu/~enron/, and download the subsets of the data at http://mng.bz/WxYg. The subsets and data collection process are described in more detail at http://www2.aueb.gr/users/ion/docs/ceas2006_paper.pdf.)

This is a dataset of emails, including both ham (extracted from the original Enron dataset using personal messages of three Enron employees) and spam emails.

What do the results tell you?

In [ ]:
test_spam_list = read_in("enron2/spam/")
print(len(test_spam_list))
print(test_spam_list[0])
test_ham_list = read_in("enron2/ham/")
print(len(test_ham_list))
print(test_ham_list[0])

test_emails = [(email_content, "spam") for email_content in test_spam_list]
test_emails += [(email_content, "ham") for email_content in test_ham_list]
random.shuffle(test_emails)

new_test_set = [(get_features(email), label) for (email, label) in test_emails]

evaluate(train_set, new_test_set, classifier)

1496
Subject: the credit law is on your side jm! Get perfect credit now!
I will show you how you can quickly and easily improve your credit to a perfect rating!
Click here now for full free details!

4361
Subject: credit model status update
Bill,
I am forwarding a memo from vincent tang about the new credit model.
English may be imperfect but the message is clear.
Vince
- - - - - - - - - - - - - - - - - - - - - - forwarded by vince j kaminski/hou/ect on 02/10/2000
02: 31 pm - - - - - - - - - - - - - - - - - - - - - - - - - - -
Vincent tang
02/08/2000 02: 43 pm
To: vince j kaminski/hou/ect@ ect, grant masson/hou/ect@ ect, tanya
Tamarchenko/hou/ect@ ect
Cc:
Subject: credit model status update
Accuracy on the training set = 0.9659173313995649
Accuracy on the test set = 0.7766774799385351
Most Informative Features
                    2004 = True             spam : ham    =    130.9 : 1.0
            prescription = True             spam : ham    =    121.3 : 1.0
                     nom = T

Combine the two datasets:

In [ ]:
spam_list = read_in("enron1/spam/") + read_in("enron2/spam/")
print(len(spam_list))
ham_list = read_in("enron1/ham/") + read_in("enron2/ham/")
print(len(ham_list))

all_emails = [(email_content, "spam") for email_content in spam_list]
all_emails += [(email_content, "ham") for email_content in ham_list]
random.shuffle(test_emails)

all_features = [(get_features(email), label) for (email, label) in all_emails]
print(len(all_features))

train_set, test_set, classifier = train(all_features, 0.8)
evaluate(train_set, new_test_set, classifier)

2996
8033
11029
Training set size = 8823 emails
Test set size = 2206 emails
Accuracy on the training set = 0.9820922588688654
Accuracy on the test set = 0.9730237322861534
Most Informative Features
                   meter = True              ham : spam   =    264.5 : 1.0
                    2004 = True             spam : ham    =    245.7 : 1.0
                   vince = True              ham : spam   =    200.8 : 1.0
                     nom = True              ham : spam   =    196.3 : 1.0
                     sex = True             spam : ham    =    195.1 : 1.0
                     ect = True              ham : spam   =    175.5 : 1.0
            prescription = True             spam : ham    =    169.2 : 1.0
                    spam = True             spam : ham    =    145.8 : 1.0
               forwarded = True              ham : spam   =    135.7 : 1.0
                     fyi = True              ham : spam   =    133.9 : 1.0
              nomination = True              ham : s